# Chapter 4. Iterators and Generators

Iteration is one of Python’s strongest features.  
At a high level, you might simply view iteration as a way to process items in a sequence.  
However, there is so much more that is possible, such as creating your own iterator objects, applying useful iteration patterns in the itertools module, making generator functions, and so forth.  
This chapter aims to address common problems involving iteration.

## [Manually Consuming an Iterator](http://chimera.labs.oreilly.com/books/1230000000393/ch04.html#_manually_consuming_an_iterator)

### Problem

You need to process items in an iterable, but for whatever reason, you can't or don't want to use a for loop.

### Solution

To manually consume an iterable, use the `next()` function and write your code to catch the `StopIteration` exception.  
This example manually reads lines from a file:

In [26]:
with open('python_ipsum.txt') as f:
    try:
        while True:
            line = next(f)
            print(line, end='')
    except StopIteration:
        pass
f.close()

Python Ipsum: Your source for Python-flavored placeholder text.
http://pythonipsum.com/

Lambda raspberrypi beautiful test script. Kwargs integration itertools dict reduce egg import cython.

Django integration functools unit object kwargs functools dictionary cython. Cython integration exception. Lambda integration diversity bdfl. Return integration exception self dunder. Python integration mercurial bdfl python lambda generator. Kwargs raspberrypi decorator unit cython import. Cython raspberrypi exception unit future klass exception. Python integration community. Object raspberrypi community bdfl cython import method.

Method raspberrypi diversity 2to3 return yield unit yield guido. Method integration mercurial unit import python exception dictionary. Django raspberrypi functools self import. Python integration mercurial dict return klass. Lambda integration mercurial 2to3 cython zen.

Import raspberrypi community pypi reduce dunder pyladies functools. Lambda raspberrypi decorator bd

Normally, `StopIteration` is used to signal the end of iteration.  
However, if you're using `next()` manually (as shown), you can also instruct it to return a terminating value, such as `None`, instead.

In [27]:
with open('python_ipsum.txt') as f:
    while True:
        line = next(f, None)
        if line is None:
            break
        print(line, end='')
f.close()

Python Ipsum: Your source for Python-flavored placeholder text.
http://pythonipsum.com/

Lambda raspberrypi beautiful test script. Kwargs integration itertools dict reduce egg import cython.

Django integration functools unit object kwargs functools dictionary cython. Cython integration exception. Lambda integration diversity bdfl. Return integration exception self dunder. Python integration mercurial bdfl python lambda generator. Kwargs raspberrypi decorator unit cython import. Cython raspberrypi exception unit future klass exception. Python integration community. Object raspberrypi community bdfl cython import method.

Method raspberrypi diversity 2to3 return yield unit yield guido. Method integration mercurial unit import python exception dictionary. Django raspberrypi functools self import. Python integration mercurial dict return klass. Lambda integration mercurial 2to3 cython zen.

Import raspberrypi community pypi reduce dunder pyladies functools. Lambda raspberrypi decorator bd

### Discussion

In most cases, the `for` statement is used to consume an iterable.  
However, every now and then, a problem calls for more precise control over the underlying iteration mechanism.  
Thus, it is useful to know what actually happens.  
The following example illustrates the basic mechanics of what happens during iteration:

In [28]:
items = [1, 2, 3]
# Get the iterator:
it = iter(items)  # Invokes items.__iter__()
# Run the iterator:
next(it)          # Invokes it.__next__()

1

In [29]:
next(it)

2

In [30]:
next(it)

3

Subsequent recipes in this chapter expand on iteration techniques, and knowledge of the basic iterator protocol is assumed.  
Be sure to tuck this first recipe away in your memory.

## [Delegating Iteration](http://chimera.labs.oreilly.com/books/1230000000393/ch04.html#delegate_iteration)

### Problem

You have built a custom container object that internally holds a list, tuple, or some other iterable.  
You would like to make iteration work with your new container.

### Solution

Typically, all you need to do is define an `__iter__()` method that delegates iteration to the internally held container.

In [31]:
class Node:
    
    def __init__(self, value):
        self._value = value
        self._children = []
        
    def __repr__(self):
        return 'Node({!r})'.format(self._value)
    
    def add_child(self, node):
        self._children.append(node)
        
    def __iter__(self):
        return iter(self._children)

Now for a demonstration:

In [32]:
if __name__ == '__main__':
    root = Node(0)
    child1 = Node(1)
    child2 = Node(2)
    root.add_child(child1)
    root.add_child(child2)
    for ch in root:
        print(ch)

Node(1)
Node(2)


In this code, the `__iter__()` method simply forwards the iteration request to the internally held `_children` attribute.

### Discussion

Python’s iterator protocol requires `__iter__()` to return a special iterator object that implements a `__next__()` method to carry out the actual iteration.  
If all you are doing is iterating over the contents of another container, you don’t really need to worry about the underlying details of how it works.  
All you need to do is to forward the iteration request along.  
The use of the `iter()` function here is a bit of a shortcut that cleans up the code.  
`iter(s)` simply returns the underlying iterator by calling `s.__iter__()`, much in the same way that `len(s)` invokes `s.__len__()`.

## [Creating New Iteration Patterns with Generators](http://chimera.labs.oreilly.com/books/1230000000393/ch04.html#_problem_59)

### Problem

You want to implement a custom iteration pattern that’s different than the usual built-in functions (e.g., `range()`, `reversed()`, etc.).

### Solution

If you want to implement a new kind of iteration pattern, define it using a generator function.  
Here’s a generator that produces a range of floating-point numbers:

In [33]:
def frange(start, stop, increment):
    x = start
    while x < stop:
        yield x
        x += increment

To use such a function, you iterate over it using a for loop or use it with some other function that consumes an iterable (e.g., `sum()`, `list()`, etc.).

In [34]:
for n in frange(0, 4, 0.5):
    print(n)

0
0.5
1.0
1.5
2.0
2.5
3.0
3.5


In [35]:
# I want to try this with a list comprehension:
comp = [n for n in frange(0, 4, 0.5)]
print(comp)

[0, 0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5]


In [36]:
list(frange(0, 1, 0.125))

[0, 0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875]

### Discussion

The mere presence of the yield statement in a function turns it into a generator.  
Unlike a normal function, a generator only runs in response to iteration.  
Here’s an experiment you can try to see the underlying mechanics of how such a function works:

In [37]:
def countdown(n):
    print('Starting to count from', n)
    while n > 0:
        yield n
        n -= 1
    print('Done, sucka!')

Create the generator and notice that no output appears:

In [38]:
c = countdown(3)
c

<generator object countdown at 0x1108bf620>

Run the program to the first `yield` and emit a value.

In [39]:
next(c)

Starting to count from 3


3

Run the program to the next `yield` and repeat:

In [40]:
next(c)

2

In [41]:
next(c)

1

The key feature is that a generator function only runs in response to "next" operations carried out in iteration.  
Once a generator function returns, iteration stops.  
However, the `for` statement that’s usually used to iterate takes care of these details, so you don’t normally need to worry about them.

## [Implementing the Iterator Protocol](http://chimera.labs.oreilly.com/books/1230000000393/ch04.html#_implementing_the_iterator_protocol)